#### Imports

In [ ]:

#Preprocesado de datos / carga de datos
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Modelo
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

# Evaluación del modelo
from tensorflow.keras.models import load_model

#### Data

In [ ]:
base_dir = "data"
train_dir = os.path.join(base_dir, 'train-cat-rabbit')
test_dir = os.path.join(base_dir, 'test-images')
valid_dir = os.path.join(base_dir, 'val-cat-rabbit')

#### Técnicas de Escalamiento y Preprocesado

In [ ]:
train_datagen = ImageDataGenerator(
#Técnicas de preprocesamiento de datos para mejorar la categorización del modelo
				rescale = 1./255, 
				rotation_range = 20, 
				width_shift_range = 0.1, 
    			height_shift_range=0.1,
				zoom_range = 0.15,
				horizontal_flip = True,
				brightness_range=[0.5, 1.2] 
    )

validation_datagen = ImageDataGenerator(rescale = 1./255) 

test_datagen = ImageDataGenerator(rescale = 1./255,)        # Escala los píxeles de las imágenes de prueba al rango [0, 1]


In [ ]:

images_size = (150, 150) #Medida de las imágenes

train_generator = train_datagen.flow_from_directory(
							train_dir,
							target_size = images_size, # Tamaño al que se redimensionan las imágenes
							batch_size = 32,  # Tamaño del lote de imágenes por batch
							class_mode ='binary', # Tipo de etiquetas: 'binary' porque hay dos clases (gato y conejo)
       						shuffle = True 
							)


test_generator = test_datagen.flow_from_directory(
							test_dir,
							target_size = images_size,
							batch_size = 32,  
							class_mode ='binary',
       						shuffle = False		
   							)


validation_generator = validation_datagen.flow_from_directory(
							valid_dir,
							target_size = images_size, 
							batch_size = 32, 
							class_mode ='binary', 
							shuffle = False
							)


plt.figure()
f, axarr = plt.subplots(1, 5, figsize=(30, 8))

for i in range(5) :
  	axarr[i].imshow(train_generator[0][0][0])

In [ ]:
# Visualizar imágenes y las etiquetas despúes de escalamiento y preprocesado
images, labels = train_generator[0]
class_names = list(train_generator.class_indices.keys())

print(images.shape)
print(labels)

plt.figure()
f, axarr = plt.subplots(1, 10, figsize=(30, 7))

for i in range(10):
    axarr[i].imshow(images[i])
    axarr[i].set_title(class_names[int(labels[i])], fontsize=15)


#### Modelo

In [ ]:
# Crear el modelo CNN

model = Sequential([
    # Capa 1: Convolucional con 32 filtros de 3x3, función de activación ReLU, y tamaño de entrada 150x150x3 
    # Capa de MaxPooling para redicir las dimensiones de la salida
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    # Capa 2: Convolucional con 64 filtros de 3x3, función de activación ReLU 
    # Capa de MaxPooling para redicir las dimensiones de la salida    
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    #Capa 3: Convolucional con 128 filtros de 3x3, función de activación ReLU
    #Capa de MaxPooling para redicir las dimensiones de la salida
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    #Aplanar la salida para pasarla a capas más densas
    Flatten(),
    
    #Poner en 0 de forma aleatoria un porcentaje de neuronas durante cada paso de entrenamiento
    Dropout(0.3),
    
    #Capa densa con 128 neuronas y función de activación ReLU
    #Capa de salida con una neurona de activación sigmoide para clasificar los gatos y conejos
    Dense(128, activation= "relu"),
    Dense(1, activation='sigmoid') 
])

# Compilar el modelo con optimizador Adam, pérdida binaria y métrica de precisión
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Mostrar resumen de la arquitectura del modelo
model.summary()

In [ ]:
history = model.fit(
        train_generator,              # Conjunto de entrenamiento con data-augmentation
        epochs=15,                    
        validation_data = validation_generator,   # Conjunto de validación para monitoreo
        verbose=1                     # Muestra el progreso en pantalla
)


In [ ]:
# Guardar el modelo 
model.save('segundo_modelo.h5')

#### Evaluar el modelo

In [ ]:
model = load_model('segundo_modelo.h5')

In [ ]:
# Evaluar el modelo
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy:.4f}')


In [ ]:
# Obtener predicciones
y_pred_probs = model.predict(test_generator)
y_pred = (y_pred_probs > 0.5).astype(int).reshape(-1)

# Etiquetas verdaderas
y_true = test_generator.classes
# Nombres de clases
class_labels = list(test_generator.class_indices.keys())

# Reporte de métricas
print("Reporte:\n")
print(classification_report(y_true, y_pred, target_names=class_labels))

In [ ]:
# Generar matriz de confusión
cm = confusion_matrix(y_true, y_pred)

# Mostrar matriz de confusión 
cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=list(train_generator.class_indices.keys()))
cm_display.plot(cmap='Blues')
plt.title("Matriz de Confusión")
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

# Accuracy

plt.figure()
plt.plot(epochs,acc,'bo',label='train accuracy')
plt.plot(epochs,val_acc,'r',label='val accuracy')
plt.title('train and val accuracy')
plt.legend()
plt.show()

# Loss
plt.figure()
plt.plot(epochs,loss, 'bo', label ='training loss')
plt.plot(epochs, val_loss, 'r', label ='validation loss')
plt.title('train and val loss')
plt.legend()

plt.show()